Run next cell only once


In [ ]:
!pip install newspaper3k
!pip install sumy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 67.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=d9e3cc123843c4a983b34569797de1388d25926e12c99d6daaf204c0674691e8
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3339 sha256=c7dd08193fbb02628feb1df9844b3af440b686ec6f56b42134d892e25e41b2be
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1

Start running here

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import newspaper
from requests.exceptions import Timeout
from textblob import TextBlob

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer


# Download the VADER lexicon
nltk.download('vader_lexicon')

def summarize_text(text, max_sentences=3):
    # Create a parser using PlaintextParser
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    # Create an LSA summarizer
    summarizer = LsaSummarizer()
    # Summarize the text
    summary = summarizer(parser.document, max_sentences)
    # Return the summary as a string
    return ' '.join(str(sentence) for sentence in summary)

# Initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()

def get_sentiment(text):
    sentiment_scores = sia.polarity_scores(text)
    return sentiment_scores['compound']

def web_scrape_clients(clients, max_number_news=5):
    news_dfs = []

    for client in clients:
        # Remove commas and replace spaces with plus signs
        client = client.replace(",", "").replace(" ", "+")

        url = f"https://www.bing.com/news/search?q={client}+-msn"
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")

            headers = [a.text for a in soup.select(".t_t a.title")]
            journals = [a["data-author"] for a in soup.select(".t_t a.title")]
            links = [a["href"] for a in soup.select(".t_t a.title")]
            snippet = [s.text for s in soup.select(".snippet")]
            summary = []
            sentiment = []

            num_news = min(len(headers), max_number_news)

            for link in links:

                try:
                    # Use requests to download the page content with a timeout of 5 seconds
                    response = requests.get(link, timeout=20)
                    page_content = response.content

                    article = newspaper.Article(link)
                    article.download(input_html=page_content)
                    article.parse()
                    article.nlp()

                    if not article.text:
                        sentiment.append("-")
                        summary.append("Could not get the summary for this article: Subscription needed.")
                        continue  # Skip articles with empty text

                    sentiment.append(get_sentiment(article.text))
                    summary.append(summarize_text(article.text))

                    if len(summary) == max_number_news:
                        break  # Stop when desired number of non-empty summaries is reached

                except Timeout:
                    summary.append("error")
                except Exception as e:
                    summary.append("error")


            news_df = pd.DataFrame({
                "Client": [client] * num_news,
                "Header": headers[:num_news],
                "Snippet": snippet[:num_news],
                "Journal": journals[:num_news],
                "Link": links[:num_news],
                "Summary" : summary[:num_news],
                "Sentiment": sentiment[:num_news]
            })

            news_dfs.append(news_df)

    final_news_df = pd.concat(news_dfs, ignore_index=True)
    final_news_df["Client"] = final_news_df["Client"].str.replace("+", " ")
    final_news_df["Link"] = final_news_df["Link"].apply(lambda x: f'<a href="{x}" target="_blank">{x}</a>')



    return final_news_df


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


After running the following line, it will ask you to type a client name and press enter. Once the table loads, you can click on the little table icon on the right to read the summaries better

In [ ]:
def get_user_input():
    clients = input("Enter a list of clients separated by commas: ")
    clients = clients.split(",")
    clients = [client.strip() for client in clients]
    return clients

def main():
    clients = get_user_input()
    final_news_df = web_scrape_clients(clients, max_number_news=5)

    # Display the DataFrame
    from IPython.display import display, HTML
    # Display the DataFrame with clickable hyperlinks
    display(HTML(final_news_df.to_html(escape=False)))
    #display(final_news_df)
    #final_news_df.to_csv('news_data.csv', index=False)  # Save as CSV

if __name__ == "__main__":
    main()


Enter a list of clients separated by commas: Hiscox, Beazley


<ipython-input-3-7f3e4b171efd>:100: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  final_news_df["Client"] = final_news_df["Client"].str.replace("+", " ")


,Client,Header,Snippet,Journal,Link,Summary,Sentiment
0,Hiscox,Hiscox Ltd (LON:HSX) is a favorite amongst institutional investors who own 72%,"If you want to know who really controls Hiscox Ltd ( LON:HSX ), then you'll have to look at the makeup of its share registry.",YAHOO!Finance,https://finance.yahoo.com/news/hiscox-ltd-lon-hsx-favorite-093008386.html,"Institutional investors own over 50% of the company, so together than can probably strongly influence board decisions. We've identified 1 warning sign with Hiscox , and understanding them should be part of your investment process. If you would prefer discover what analysts are predicting in terms of future growth, do not miss this free report on analyst forecasts.",0.9959
1,Hiscox,Hiscox Ltd: Syndicates 33 and 6104 – results and estimates,"Syndicates 33 and 6104 - results and estimates Hamilton, Bermuda (10 November 2023) - Hiscox Ltd (LSE: HSX), the ...",Cbonds,https://cbonds.com/news/2578985/,"Full data on over 800,000 bonds and stocks worldwide Powerful bond screener Ratings from the top 3 global ratings agencies, plus over 70 local ones",0.5574
2,Hiscox,Hiscox Ltd HSX,Morningstar Quantitative Ratings for Stocks are generated using an algorithm that compares companies that are not under analyst coverage to peer companies that do receive analyst-driven ratings ...,Morningstar,https://www.morningstar.com/stocks/xlon/hsx/quote,"Companies with quantitative ratings are not formally covered by a Morningstar analyst, but are statistically matched to analyst-rated companies, allowing our models to calculate a quantitative moat, fair value, and uncertainty rating. Is it the right time to buy, sell, or hold? Start a free trial of Morningstar Investor to unlock exclusive ratings and continuous analyst coverage to help you decide if HSX is a good fit for your portfolio.",0.9628
3,Hiscox,Hiscox reports cyber-attacks continue to rise year-on-year,"UK insurer, Hiscox has released its seventh annual Cyber Readiness Report. The key finding from the report was that 53% of ...",enterprisetimes.co.uk,https://www.enterprisetimes.co.uk/2023/11/02/hiscox-reports-cyber-attacks-continue-to-rise-year-on-year/,"This website is using a security service to protect itself from online attacks. There are several actions that could trigger this block including submitting a certain word or phrase, a SQL command or malformed data. Please include what you were doing when this page came up and the Cloudflare Ray ID found at the bottom of this page.",0.7450
4,Hiscox,Hiscox reports strong written premium growth,Specialist insurer Hiscox reported strong growth in group insurance contract written premiums (ICWP) on Wednesday in the ...,Sharecast,https://www.sharecast.com/news/news-and-announcements/hiscox-reports-strong-written-premium-growth--15248966.html,"The London Market segment of Hiscox saw net ICWP increase by an impressive 18.1% to reach $676.7m. Furthermore, Hiscox ILS funds delivered record performance, generating an increasing fee income for the group. Despite an active third quarter, aggregate natural catastrophe losses year-to-date remained within budget.",0.9985
5,Beazley,"Beazley cyber cat bond hits the market, $75m PoleStar Re Ltd.","Beazley, the London headquartered specialty insurance and reinsurance underwriter, has now entered the 144A catastrophe bond ...",Artemis,https://www.artemis.bm/news/beazley-cyber-cat-bond-hits-the-market-75m-polestar-re-ltd/,"Beazley had already sponsored three private cyber catastrophe bonds so far this year, but we were told that a renewal of the coverage would likely come in full 144a cat bond form, and this new PoleStar Re cyber cat bond appears to be that deal. With the AXIS Capital deal progressing and expected to be priced in the next day or so, this sends a strong signal that insurance-linked securities (ILS) investors are welcoming this new peril and ready to assess it, investing in it where appropriate for their portfolios. In addition, Gallagher Securiti